In [ ]:
from srai.embedders import Hex2VecEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders import OSMOnlineLoader
from srai.neighbourhoods import H3Neighbourhood
from srai.regionalizers import H3Regionalizer, geocode_to_region_gdf
from srai.plotting import plot_regions, plot_numeric_data
from pytorch_lightning import seed_everything

In [ ]:
SEED = 71
seed_everything(SEED)

### Load data from OSM

First use geocoding to get the area

In [ ]:
area_gdf = geocode_to_region_gdf("Wrocław, Poland")
plot_regions(area_gdf, tiles_style="CartoDB positron")

Next, download the data for the selected region and the specified tags. We're using `OSMOnlineLoader` here, as it's faster for low numbers of tags. In a real life scenario with more tags, you would likely want to use the `OSMPbfLoader`.

In [ ]:
tags = {
    "leisure": "park",
    "landuse": "forest",
    "amenity": ["bar", "restaurant", "cafe"],
    "water": "river",
    "sport": "soccer",
}
loader = OSMOnlineLoader()

features_gdf = loader.load(area_gdf, tags)

folium_map = plot_regions(area_gdf, colormap=["rgba(0,0,0,0)"], tiles_style="CartoDB positron")
features_gdf.explore(m=folium_map)

## Prepare the data for embedding

After downloading the data, we need to prepare it for embedding. Namely - we need to regionalize the selected area, and join the features with regions.

In [ ]:
regionalizer = H3Regionalizer(resolution=9)
regions_gdf = regionalizer.transform(area_gdf)
plot_regions(regions_gdf, tiles_style="CartoDB positron")

In [ ]:
joiner = IntersectionJoiner()
joint_gdf = joiner.transform(regions_gdf, features_gdf)
joint_gdf

## Embedding

After preparing the data we can proceed with generating embeddings for the regions.

In [ ]:
import warnings

neighbourhood = H3Neighbourhood(regions_gdf)
embedder = Hex2VecEmbedder([15, 10])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    embeddings = embedder.fit_transform(
        regions_gdf,
        features_gdf,
        joint_gdf,
        neighbourhood,
        trainer_kwargs={"max_epochs": 5, "accelerator": "cpu"},
        batch_size=100,
    )
embeddings

### Visualizing the embeddings' similarity

In [ ]:
from sklearn.cluster import KMeans

clusterizer = KMeans(n_clusters=5, random_state=SEED)
clusterizer.fit(embeddings)

embeddings["cluster"] = clusterizer.labels_
embeddings

In [ ]:
plot_numeric_data(regions_gdf, embeddings, "cluster", tiles_style="CartoDB positron")